Make sure the transformer version is up to data

In [1]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection, AutoModelForCausalLM


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# load the caption from the txt file
with open("OFA_output.txt", "r") as f:
    caption = f.read()


# DINO
dino_processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
dino_model = AutoModelForZeroShotObjectDetection.from_pretrained("IDEA-Research/grounding-dino-tiny")

image_path = "/Users/junwu/Documents/ECS270/Project/Image-Model-with-Grounding-DINO/input-data/clock.jpg"
image = Image.open(image_path)

# Use OFA caption as input to DINO
dino_inputs = dino_processor(images=image, text=caption, return_tensors="pt")
with torch.no_grad():
    dino_outputs = dino_model(**dino_inputs)

# Post-process and display results =
results = dino_processor.post_process_grounded_object_detection(
    dino_outputs,
    dino_inputs.input_ids,
    box_threshold=0.4,
    text_threshold=0.3,
    target_sizes=[image.size[::-1]]
)

for box, score, label in zip(results[0]["boxes"], results[0]["scores"], results[0]["labels"]):
    box = [round(x, 2) for x in box.tolist()]
    print(f"Detected {label} with confidence {round(score.item(), 3)} at location {box}")

Detected clock with confidence 0.581 at location [121.48, 26.56, 490.35, 388.6]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/models/grounding_dino/processing_grounding_dino.py:95: FutureWarning: The key `labels` is will return integer ids in `GroundingDinoProcessor.post_process_grounded_object_detection` output since v4.51.0. Use `text_labels` instead to retrieve string object names.
  warnings.warn(self.message, FutureWarning)
